In [2]:
import json

f = open('../Volumes/Aux/Downloaded/Data-Upload/FB60K+NYT10/text/train.json')
data_corpus = json.load(f)

f = open('../Volumes/Aux/Downloaded/Data-Upload/FB60K+NYT10/text/test.json')
tmp = json.load(f)

data_corpus = data_corpus + tmp

In [10]:
corpus_filename = 'nyt10_corpus.json'
chunked_filename = 'nyt10_corpus_chunked.json'

with open(corpus_filename) as corpus_file:

    with open(chunked_filename, "w") as chunked_file:
        for jline in json.load(corpus_file):
            print(jline)
            break

{'sentence': 'sen. charles e. schumer called on federal safety officials yesterday to reopen their investigation into the fatal crash of a passenger jet in belle_harbor , queens , because equipment failure , not pilot error , might have been the cause . ###END###\n', 'head': {'word': 'queens', 'id': 'm.0ccvx'}, 'tail': {'word': 'belle_harbor', 'id': 'm.05gf08'}, 'relation': '/location/location/contains'}


In [ ]:
from transformers import RagTokenizer, RagRetriever, RagModel
import torch

tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
retriever = RagRetriever.from_pretrained(
    "facebook/rag-token-base", index_name="exact", use_dummy_dataset=True
)
# initialize with RagRetriever to do everything in one forward call
model = RagModel.from_pretrained("facebook/rag-token-base", retriever=retriever)

inputs = tokenizer("How many people live in Paris?", return_tensors="pt")

input_ids = inputs["input_ids"]
# 1. Encode
question_hidden_states = model.question_encoder(input_ids)[0]
# 2. Retrieve
docs_dict = retriever(input_ids.numpy(), question_hidden_states.detach().numpy(), return_tensors="pt")
doc_scores = torch.bmm(
    question_hidden_states.unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2)
).squeeze(1)

# or directly generate
generated = model.generate(
    input_ids = inputs["input_ids"],
    context_input_ids=docs_dict["context_input_ids"],
    context_attention_mask=docs_dict["context_attention_mask"],
    doc_scores=doc_scores,
)
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)

In [ ]:
from datasets import load_dataset

wiki_dataset = load_dataset("wikipedia", "20220301.en")

In [ ]:
wiki_dataset

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
torch.set_grad_enabled(False)
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

In [ ]:
ds_with_embeddings = wiki_dataset.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["text"],truncation=True, padding="longest", return_tensors="pt"))[0][0].numpy()})